In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import shutil
from pydub import AudioSegment
from pydub.utils import make_chunks
import librosa
import matplotlib.pyplot as plt
import librosa.display
import soundfile as sd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
%matplotlib inline
SEED = 786
random.seed(SEED)

In [ ]:
# Function to split the input audio files from directory into the desired duration of the chunk

dataset_folder_name = '5_speakers_audio_data'
def split_audio_file(initial_directory, final_directory, duration):#enter duration in milliseconds
    for folder in os.listdir(os.path.join(initial_directory,dataset_folder_name)):
        for file in os.listdir(os.path.join(initial_directory,dataset_folder_name,folder)):
            
            filepath = os.path.join(initial_directory, '5_speakers_audio_data', folder, file);
            filename_prefix = os.path.splitext(os.path.basename(file))[0]
            myaudio = AudioSegment.from_file(filepath, "wav") 
            chunk_length_ms = duration # pydub calculates in millisec
            chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of 3 sec

            #Export all of the individual chunks as wav files

            for i, chunk in enumerate(chunks):
                chunk_name = final_directory+"/"+filename_prefix+"_chunk{0}.wav".format(i)
                print(chunk_name)
                chunk.export(chunk_name, format="wav")

In [ ]:
dir = ('/home/agrawal.shau/audio_dataset')
dir_final = ('/home/agrawal.shau/audio_dataset/5_speakers_audio_data_3sec')
duration = 3000

split_audio_file(dir, dir_final, duration)

In [ ]:
# Generate the mfccs as pyplot and saving the plot as .jpg format file in the directory

audio_file_folder_name = '5_speakers_audio_data_3sec'

def generate_mfcc(intial_directory, final_directory, no_of_coefficients):
    for file in os.listdir(os.path.join(initial_directory,audio_file_folder_name)):
        
        filepath = initial_directory+'/'+audio_file_folder_name+'/'+file;
        mfcc_image_prefix = os.path.splitext(os.path.basename(file))[0]
        #filepath = os.path.join(initial_directory, audio_file_folder_name, file);
        print("Currently processing ", file)

        y, sr = librosa.load(filepath)
        mfccs = librosa.feature.mfcc(y, n_mfcc = no_of_coefficients, sr = sr)
        #plt.figure(figsize = (25,10))
        librosa.display.specshow(mfccs, x_axis = 'time', sr = sr)
        fig = plt.gcf()
        for i, mfccs in enumerate(mfccs):
            
            fig.savefig(final_directory+"/"+mfcc_image_prefix+'.jpg')

In [ ]:
initial_directory = ('/home/agrawal.shau/audio_dataset')
final_directory = ('/home/agrawal.shau/audio_dataset/mfccs_3sec')
generate_mfcc(initial_directory, final_directory, 13)

In [ ]:
# This is to sort the mfcc .jpg files of each of the 5 speakers in train, test and validation dataset
def sort_files(directory, speaker_0000_files, speaker_0001_files, speaker_0002_files, speaker_0003_files, speaker_0004_files):   

    for file in os.listdir(directory):
        filename = os.path.splitext(os.path.basename(file))[0]
        if filename.startswith('Speaker_0000'):
            speaker_0000_files.append(file) # for speaker 0
        if filename.startswith('Speaker_0001'):
            speaker_0001_files.append(file) # for speaker 1
        if filename.startswith('Speaker_0002'):
            speaker_0002_files.append(file) # for speaker 2
        if filename.startswith('Speaker_0003'):
            speaker_0003_files.append(file) # for speaker 3
        if filename.startswith('Speaker_0004'):
            speaker_0004_files.append(file) # for speaker 5

In [ ]:
directory = '/home/agrawal.shau/audio_dataset/mfccs_3sec'
speaker_0000_files = []
speaker_0001_files = []
speaker_0002_files = []
speaker_0003_files = []
speaker_0004_files = []
sort_files(directory,speaker_0000_files, speaker_0001_files, speaker_0002_files, speaker_0003_files, speaker_0004_files)
len(speaker_0000_files), len(speaker_0001_files), len(speaker_0002_files), len(speaker_0003_files), len(speaker_0004_files)

In [ ]:
np.random.shuffle(speaker_0000_files)
np.random.shuffle(speaker_0001_files)
np.random.shuffle(speaker_0002_files)
np.random.shuffle(speaker_0003_files)
np.random.shuffle(speaker_0004_files)

Preparing Training, Validation and Test Set

In [ ]:
# Preparing Training set
speaker_0000_train = np.random.choice(speaker_0000_files, size = 230, replace = False)
speaker_0001_train = np.random.choice(speaker_0001_files, size = 230, replace = False)
speaker_0002_train = np.random.choice(speaker_0002_files, size = 230, replace = False)
speaker_0003_train = np.random.choice(speaker_0003_files, size = 230, replace = False)
speaker_0004_train = np.random.choice(speaker_0004_files, size = 230, replace = False)
speaker_0000_files = list(set(speaker_0000_files) - set(speaker_0000_train))
speaker_0001_files = list(set(speaker_0001_files) - set(speaker_0001_train))
speaker_0002_files = list(set(speaker_0002_files) - set(speaker_0002_train))
speaker_0003_files = list(set(speaker_0003_files) - set(speaker_0003_train))
speaker_0004_files = list(set(speaker_0004_files) - set(speaker_0004_train))

In [ ]:
# Preparing validation set 
speaker_0000_val = np.random.choice(speaker_0000_files, size = 30, replace = False)
speaker_0001_val = np.random.choice(speaker_0001_files, size = 30, replace = False)
speaker_0002_val = np.random.choice(speaker_0002_files, size = 30, replace = False)
speaker_0003_val = np.random.choice(speaker_0003_files, size = 30, replace = False)
speaker_0004_val = np.random.choice(speaker_0004_files, size = 30, replace = False)
speaker_0000_files = list(set(speaker_0000_files) - set(speaker_0000_val))
speaker_0001_files = list(set(speaker_0001_files) - set(speaker_0001_val))
speaker_0002_files = list(set(speaker_0002_files) - set(speaker_0002_val))
speaker_0003_files = list(set(speaker_0003_files) - set(speaker_0003_val))
speaker_0004_files = list(set(speaker_0004_files) - set(speaker_0004_val))

In [ ]:
# Test Set
speaker_0000_test = speaker_0000_files
speaker_0001_test = speaker_0001_files
speaker_0002_test = speaker_0002_files
speaker_0003_test = speaker_0003_files
speaker_0004_test = speaker_0004_files

Labelling train, validation and test set

In [ ]:
Img_Width = 300
Img_Height = 300
Img_Size = (Img_Width, Img_Height)
train_imgs = [*speaker_0000_train, *speaker_0001_train, *speaker_0002_train, *speaker_0003_train, *speaker_0004_train]
val_imgs = [*speaker_0000_val, *speaker_0001_val, *speaker_0002_val, *speaker_0003_val, *speaker_0004_val]
train_files_with_labels = []
for file in train_imgs:
    temp_list  = [file, int(file[11])];
    train_files_with_labels.append(temp_list);
train_files_with_labels = pd.DataFrame(train_files_with_labels # generating label for train set
train_files_with_labels.to_csv('/home/agrawal.shau/audio_dataset/train.csv');
val_files_with_labels = []
for file in val_imgs:
    temp_list  = [file, int(file[11])];
    val_files_with_labels.append(temp_list);
val_files_with_labels = pd.DataFrame(val_files_with_labels) # generating label for validation set
val_files_with_labels.to_csv('/home/agrawal.shau/audio_dataset/validation.csv');
print(f"Number of train images = {len(train_imgs)} \nValidation images = {len(val_imgs)}")

In [ ]:
test_imgs = [*speaker_0000_test, *speaker_0001_test, *speaker_0002_test, *speaker_0003_test, *speaker_0004_test]
test_files_with_labels = []
for file in test_imgs:
    temp_list  = [file, int(file[11])];
    test_files_with_labels.append(temp_list);
test_files_with_labels = pd.DataFrame(test_files_with_labels) # generating labels for test set
test_files_with_labels.to_csv('/home/agrawal.shau/audio_dataset/test.csv');
print(f"Number of Test images = {len(test_imgs)}")

In [16]:
print("Train Files Csv \n", train_files_with_labels)
print("Val Files Csv \n", val_files_with_labels)
print("Test Files Csv \n", test_files_with_labels)

Train Files Csv 
                                    0  1
0     Speaker_0000_00009_chunk20.jpg  0
1     Speaker_0000_00011_chunk15.jpg  0
2      Speaker_0000_00007_chunk0.jpg  0
3     Speaker_0000_00005_chunk18.jpg  0
4      Speaker_0000_00003_chunk8.jpg  0
...                              ... ..
1145  Speaker_0004_00012_chunk18.jpg  4
1146  Speaker_0004_00008_chunk19.jpg  4
1147  Speaker_0004_00004_chunk16.jpg  4
1148   Speaker_0004_00000_chunk3.jpg  4
1149   Speaker_0004_00013_chunk3.jpg  4

[1150 rows x 2 columns]
Val Files Csv 
                                   0  1
0    Speaker_0000_00012_chunk18.jpg  0
1     Speaker_0000_00006_chunk2.jpg  0
2    Speaker_0000_00013_chunk10.jpg  0
3     Speaker_0000_00005_chunk3.jpg  0
4     Speaker_0000_00005_chunk6.jpg  0
..                              ... ..
145  Speaker_0004_00004_chunk13.jpg  4
146   Speaker_0004_00011_chunk6.jpg  4
147   Speaker_0004_00012_chunk4.jpg  4
148  Speaker_0004_00007_chunk17.jpg  4
149   Speaker_0004_00006_chunk7.

In [ ]:
# Resizing train images
direcotry = '/home/agrawal.shau/audio_dataset/mfccs_3sec/'
resized_imgs = [img_to_array(load_img(img, target_size=Img_Size)) for img in [direcotry + img for img in train_imgs]]
train_imgs = np.array(resized_imgs)
# Resizing Validation images
direcotry = '/home/agrawal.shau/audio_dataset/mfccs_3sec/'
resized_imgs = [img_to_array(load_img(img, target_size=Img_Size)) for img in [direcotry + img for img in val_imgs]]
val_imgs = np.array(resized_imgs)

In [ ]:
# Scaling each image with values between 0 and 1
train_imgs_scaled = train_imgs.astype('float32') 
validation_imgs_scaled = val_imgs.astype('float32') 
train_imgs_scaled /= 255 
validation_imgs_scaled /= 255 
# visualize a sample image 
print(train_imgs[0].shape) 
array_to_img(train_imgs[0])

In [ ]:
data = pd.read_csv("train.csv")
train_labels = data['1'].values

data = pd.read_csv("validation.csv")
val_labels = data['1'].values

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(train_imgs,train_labels, batch_size=5)
val_generator = val_datagen.flow(val_imgs,val_labels, batch_size=5)

# Model Creation

Created the datasets, now we shall only be using train_generator and val_generator as an iterator and pass it for model training. 

In [21]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(Img_Height,Img_Width,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output)
for layer in restnet.layers[13:]:
    layer.trainable = False
restnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [22]:
restnet.layers[13:]

In [23]:
output

<KerasTensor: shape=(None, 204800) dtype=float32 (created by layer 'flatten_1')>

In [29]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu'))#, input_dim=(Img_Height,Img_Width,3)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 204800)            23587712  
                                                                 
 dense_9 (Dense)             (None, 512)               104858112 
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 5)                 2565      
                                                                 
Total params: 128,711,045
Trainable params: 105,174,27

In [30]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=230, 
                              epochs=25,
                              validation_data=val_generator, 
                              validation_steps=50, 
                              verbose=1)

<ipython-input-30-ead03c370caa>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/25
230/230 [==============================] - 315s 1s/step - loss: 2.2422 - accuracy: 0.3426 - val_loss: 1.6722 - val_accuracy: 0.2000
Epoch 2/25
230/230 [==============================] - 290s 1s/step - loss: 1.1902 - accuracy: 0.5330
Epoch 3/25
230/230 [==============================] - 293s 1s/step - loss: 0.8642 - accuracy: 0.6643
Epoch 4/25
230/230 [==============================] - 297s 1s/step - loss: 0.5948 - accuracy: 0.7861
Epoch 5/25
230/230 [==============================] - 306s 1s/step - loss: 0.5016 - accuracy: 0.8104
Epoch 6/25
230/230 [==============================] - 304s 1s/step - loss: 0.3869 - accuracy: 0.8626
Epoch 7/25
230/230 [==============================] - 304s 1s/step - loss: 0.3233 - accuracy: 0.8835
Epoch 8/25
230/230 [==============================] - 304s 1s/step - loss: 0.2455 - accuracy: 0.9096
Epoch 9/25
230/230 [==============================] - 305s 1s/step - loss: 0.2703 - accuracy: 0.9043
Epoch 10/25
230/230 [============================

In [31]:
model.save('speaker_recognition_model_resnet50')

INFO:tensorflow:Assets written to: speaker_recognition_model_resnet50/assets


/home/agrawal.shau/.local/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/agrawal.shau/.local/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [32]:
history

In [ ]:
# Saving history of the model as a pandas dataframe
hist_df = pd.DataFrame(history.history)

# Saving the pandas dataframe as csv file
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

Tensorflow has API, which can perform the model training across multiple GPUs with minimal code changes. Thus parallelizing the model training below

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [ ]:
# Viewing the list of available GPUs 
devices = tf.config.experimental.list_physical_devices(“GPU”)
# Use subset of available GPUs
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:2"]) # Using 2 GPUs

# Building model

with mirrored_strategy.scope():
     model = tf.keras.applications.resnet50.ResNet50(
             include_top=False, weights='imagenet', input_shape=(Img_Height,Img_Width,3))
  
     opt = tf.keras.optimizers.SGD(lr=1e-4)
 
     model.compile(loss='sparse categorical crossentropy', 
                   optimizer=opt, metrics=[‘accuracy’])

model.fit(train_generator, 
          steps_per_epoch=230, 
          epochs=25,
          validation_data=val_generator, 
          validation_steps=50, 
          verbose=1)